In [22]:
import gym
import numpy as np
from gym import spaces
from stable_baselines3 import PPO
import shap
from mlutils import *
from tqdm import tqdm

In [23]:
# 配置类
class Config:
    # 目标性能阈值（根据需求修改）
    TARGETS = np.array([134, 278, 22.5])  # 三个性能指标的最低要求
    # SHAP参数
    SAMPLE_RATIO = 0.1     # 背景数据采样比例
    N_SUMMARY = 100        # SHAP背景数据压缩量
    TOP_K = 5              # 每个样本选择的关键参数数量

    MAX_STEPS = 100  # 最大步数
    BUFFER_RATIO = 0.1  # 动态范围比例

In [24]:
# 识别不合格样本
def find_unqualified_samples(y,  tragets):
    """找到至少有一个性能指标不达标的样本"""
    unqualified_mask = np.any(y < tragets, axis=1)
    return np.where(unqualified_mask)[0]

# 判断力学性能是否达标（用户自定义）
def is_satisfied(y_pred,  tragets):
    # 示例：假设 y_pred 是 3 个力学性能值，targets 是达标指标
    return all(y >= t for y, t in zip(y_pred, tragets))

#反标准化函数
def inverse_normalize(sample,scaler):
    inverse_sample=scaler.inverse_transform(sample.reshape(1,-1))
    return inverse_sample[0]
    

In [25]:
Procedure_header=['化学元素含量', '化学元素含量', '化学元素含量', '化学元素含量', '化学元素含量', '化学元素含量', '化学元素含量', '化学元素含量', '化学元素含量', '热轧', '热轧', '热轧', '热轧', '热轧', '热轧', '热轧', '热轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '冷轧', '镀锌', '镀锌', '镀锌', '镀锌', '镀锌', '镀锌', '镀锌', '镀锌', '镀锌', '镀锌', '镀锌', '镀锌', '镀锌', '镀锌', '镀锌', '镀锌', '镀锌']
Parameter_header=['碳', '硅', '锰', '磷', '硫', '钛', '铌', '氧', '氮', '材料实际重量', '出口材料实际厚度', '出口材料实际宽度', '卷取温度平均值', '出炉温度', '在炉时间', '精轧入口平均温度', '精轧出口平均温度', '出口材料实际厚度公差', '出口材料实际宽度公差', '出口材料实际重量', '入口材料1厚度', '入口材料1宽度', '入口材料1重量', 'S1机架压下率', 'S2机架压下率', 'S3机架压下率', 'S4机架压下率', 'S5机架压下率', 'S1机架入口张力', 'S1~S2机架间张力', 'S2～S3机架间张力', 'S3～S4机架间张力', 'S4～S5机架间张力', 'S5出口张力', 'S1机架入口单位张力', 'S1~S2机架间单位张力', 'S2~S3机架间单位张力', 'S3~S4机架间单位张力', 'S4~S5机架间单位张力', 'S5机架工作轧辊粗糙度(底)', 'S5机架工作轧辊粗糙度(上)', '1#机架轧制力模型设定值', '2#机架轧制力模型设定值', '3#机架轧制力模型设定值', '4#机架轧制力模型设定值', '5#机架轧制力模型设定值', '拉矫率平均值', '1#酸槽温度', '2#酸槽温度', '3#酸槽温度', '酸洗工序速度平均值1', '上表面镀层重量', '下表面镀层重量', '平整率平均值', '上表面涂油量', '下表面涂油量', '工艺段速度平均值', 'ES平均温度', 'FCS平均温度', 'IHS平均温度', 'SCS平均温度', 'SF平均温度', 'RCS平均温度', 'RTF平均温度', 'JPF平均温度', 'JPF平均温度', 'JPF平均温度', 'JPF平均温度']

In [26]:
test_x,test_y=x_y_split(test_data_path, scaler=joblib.load(scaler_model_path))
train_x,train_y=x_y_split(train_data_path, scaler=joblib.load(scaler_model_path))

In [27]:
model_name='Random Forest'
model=models[model_name]
model=joblib.load(pre_model_path + model_name + '.pkl')

In [28]:
#随机提取train_x的0.1倍样本
train_x_sample=train_x[np.random.choice(train_x.shape[0], int(train_x.shape[0]*0.1), replace=False)]
train_x_sample_summary = shap.sample(train_x_sample, 100)
#使用shap的kernel explainer对混合模型
explainer = shap.KernelExplainer(model.predict, train_x_sample_summary)
def cal_shap_values(x):
    return explainer.shap_values(x)

In [29]:
def generate_param_bounds(sample, X_data, top_indices, buffer_ratio):
    """
    作用：根据历史数据和当前样本生成优化参数范围
    sample: 当前样本
    X_data: 历史数据
    top_indices: 优化参数索引
    buffer——ratio: 动态范围比例
    """
    bounds = {}
    for idx in top_indices:
        # 全局数据范围（考虑工艺限制）
        global_min = X_data[:, idx].min()
        global_max = X_data[:, idx].max()
        
        # 当前值
        current_val = sample[idx]
        
        # 动态范围：当前值±buffer_ratio范围的全局裁剪
        buffer_range = (global_max - global_min) * buffer_ratio
        min_val = max(global_min, current_val - buffer_range)
        max_val = min(global_max, current_val + buffer_range)
        
        bounds[f'x{idx}'] = (min_val, max_val)
    return bounds

In [ ]:
# 自定义环境
class ParamOptimizationEnv(gym.Env):
    def __init__(self, sample, top_indices, bounds, model, is_satisfied):
        super(ParamOptimizationEnv, self).__init__()
        self.sample = sample.copy()  # 初始工艺参数
        self.top_indices = top_indices  # 需要优化的参数索引
        self.bounds = bounds  # 优化范围
        self.model = model  # 预测模型
        self.is_satisfied = is_satisfied  # 达标判断函数
        self.current_params = sample.copy()
        self.action_space = spaces.Box(low=-1, high=1, shape=(len(top_indices),), dtype=np.float32)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(len(top_indices),), dtype=np.float32)
        self.max_steps = Config.MAX_STEPS
        self.current_step = 0

    def reset(self):
        self.current_params = self.sample.copy()
        self.current_step = 0
        return self._get_obs()

    def step(self, action):
        # 更新需要优化的参数
        for i, idx in enumerate(self.top_indices):
            delta = action[i] * (self.bounds[f'x{idx}'][1] - self.bounds[f'x{idx}'][0]) / 2
            self.current_params[idx] += delta
            self.current_params[idx] = np.clip(self.current_params[idx], self.bounds[f'x{idx}'][0], self.bounds[f'x{idx}'][1])

        # 预测力学性能
        y_pred = self.model.predict(self.current_params.reshape(1, -1))[0]
        satisfied = self.is_satisfied(y_pred, Config.TARGETS)
        
        # 奖励设计
        if satisfied:
            reward = 10  # 达标时高奖励
        else:
            # 未达标时奖励与目标距离负相关
            reward = -sum(abs(y - t) for y, t in zip(y_pred, [0.8, 0.9, 1.0])) / 3
        reward -= 0.1  # 步数惩罚
        
        self.current_step += 1
        done = satisfied or self.current_step >= self.max_steps
        return self._get_obs(), reward, done, {"y_pred": y_pred}

    def _get_obs(self):
        return self.current_params[self.top_indices]

# 主函数：优化单个样本
def optimize_sample(sample, X_data, model):
    # 计算 SHAP 值并选择 top_k 参数
    shap_values = cal_shap_values(sample)
    top_indices = np.argsort(np.abs(shap_values).mean(1))[::-1][:Config.TOP_K]
    bounds = generate_param_bounds(sample, X_data, top_indices, Config.BUFFER_RATIO)

    # 创建环境
    env = ParamOptimizationEnv(sample, top_indices, bounds, model, is_satisfied)

    # 训练 PPO 模型
    ppo_model = PPO("MlpPolicy", env, verbose=0, batch_size=256, n_epochs=10)
    ppo_model.learn(total_timesteps=100)

    # 测试优化
    obs = env.reset()
    for _ in range(Config.MAX_STEPS):
        action, _ = ppo_model.predict(obs)
        obs, reward, done, info = env.step(action)
        if done:
            # print(f"优化完成，调整后的力学性能: {info['y_pred']}")
            break
    if is_satisfied(info['y_pred'], Config.TARGETS):
        status='success'
        opt_pred=info['y_pred']
    else:
        status='fail'
        opt_pred=None
    return env.current_params, top_indices, bounds, status, opt_pred

# # # 示例调用
# unqualified_samples = find_unqualified_samples(test_y, Config.TARGETS)
# sample_idx = unqualified_samples[0]
# sample = test_x[sample_idx]
# print(f"待优化样本索引: {sample_idx}")
# optimized_params, top_indices, bounds, status,opt_pred = optimize_sample(sample, train_x_sample, model)
# print(f"优化结果: {status}")
# if status=='success':
#     print(f"优化前的力学性能: {test_y[sample_idx]}")
#     print(f"优化后的力学性能: {opt_pred}")

#     print('优化前的参数：')
#     print(inverse_normalize(sample,joblib.load(scaler_model_path)))
#     print('优化后的参数：')
#     print(inverse_normalize(optimized_params,joblib.load(scaler_model_path)))

# else:
#     print(f"优化失败")
#     print(f"优化前的力学性能: {test_y[sample_idx]}")
#     print(f"优化后的力学性能: {opt_pred}")
#     print('优化前的参数：')
#     print(inverse_normalize(sample,joblib.load(scaler_model_path)))
#     print('优化后的参数：')
#     print(inverse_normalize(optimized_params,joblib.load(scaler_model_path)))


待优化样本索引: 4
优化结果: success
优化前的力学性能: [471.  570.   21.5]
优化后的力学性能: [463.54  558.48   25.625]
优化前的参数：
[7.2999999e-02 2.0000000e-01 1.0100000e+00 8.9999996e-03 3.0000000e-03
 2.9999999e-02 4.4000000e-02 0.0000000e+00 3.8999999e-03 2.9490000e+01
 3.8000002e+00 1.4378000e+03 6.0500000e+02 1.2420000e+03 1.7500000e+02
 1.0370000e+03 8.9000000e+02 1.6380000e+00 1.4378000e+03 2.9209997e+01
 3.8000002e+00 1.4378000e+03 2.9490000e+01 1.7912001e+01 2.3148001e+01
 1.9091999e+01 1.4289999e+01 1.1330000e+00 2.6600000e+02 4.5700000e+02
 4.2100000e+02 3.9100000e+02 3.3000000e+02 1.0000000e+02 4.9000000e+01
 1.0200000e+02 1.2100000e+02 1.4000000e+02 1.3900000e+02 3.0000000e+00
 2.9999998e+00 1.8476000e+04 1.9890000e+04 1.8713000e+04 1.6991002e+04
 9.3460000e+03 6.0000002e-01 8.6000000e+01 8.2000000e+01 8.5000000e+01
 1.5500000e+02 5.7399998e+01 5.8799999e+01 1.5599999e+00 5.9900000e+02
 5.9900000e+02 7.0000000e+01 1.8300000e+02 4.3200000e+02 2.9200000e+02
 4.5400000e+02 6.8000000e+02 4.4500003e+02 7.9400

In [39]:
def optimize_many(test_x, test_y, X_data, model):
        #找到不合格样本
    unqualified_samples = find_unqualified_samples(test_y, Config.TARGETS)
    results = []
    i=0
    for idx in tqdm(unqualified_samples):
        
        i=i+1

        ori_sample = test_x[idx]
        ori_pred=test_y[idx]
        optimized_params, top_indices, bounds, status,opt_pred = optimize_sample(ori_sample, X_data, model)
        #对工艺参数进行反标准化
        ori_sample=inverse_normalize(ori_sample,scaler)
        optimized_params=inverse_normalize(optimized_params,scaler)
        #记录结果
        if status=='success':
            param_changes=[]
            for p_idx in top_indices:
                ori_val=ori_sample[p_idx]
                opt_val=optimized_params[p_idx]
                change_pct=(opt_val-ori_val)/ori_val*100
                param_changes.append({
                    'param_idx':int(p_idx),
                    'param_name':Procedure_header[int(p_idx)]+'过程的'+Parameter_header[int(p_idx)],
                    'ori_val':float(ori_val),
                    'opt_val':float(opt_val),
                    'change_pct':float(change_pct)
                
                })
            #把ori_pred和opt_pred转换为列表
            if hasattr(ori_pred,'tolist'):
                ori_pred=ori_pred.tolist()
            else:
                ori_pred=list(ori_pred)
            if hasattr(opt_pred,'tolist'):
                opt_pred=opt_pred.tolist()
            else:
                opt_pred=list(opt_pred)
            
            results.append({
                'sample_id':int(idx),
                'status':status,
                'original_performance':ori_pred,
                'optimized_performance':opt_pred,
                'param_changes':param_changes,
                'top_params_index':[int(p_idx) for p_idx in top_indices.tolist()],
                'top_paras':[Parameter_header[int(pidx)] for pidx in top_indices.tolist()]
            })
        
        else:
            results.append({
                'sample_id':int(idx),
                'status':status,
                'original_performance':ori_pred,
                'optimized_performance':None,
                'param_changes':None,
                'top_params_index':[int(p_idx) for p_idx in top_indices.tolist()],
                'top_paras':[Parameter_header[int(pidx)] for pidx in top_indices.tolist()]
            })

        if i==4:
            break
        
    df_results=pd.DataFrame(results)
    success_rate=(df_results['status'] == 'success').mean()
    print(f"\n优化完成！成功率：{success_rate:.1%}")
    # 保存到CSV（示例），编码为UTF-8
    df_results.to_csv('results_PPO.csv', index=False, encoding='utf-8-sig')
    
    return df_results


In [40]:
optimize_many(test_x, test_y, train_x_sample, model)

  1%|          | 3/408 [02:18<5:12:33, 46.31s/it]


优化完成！成功率：100.0%


,sample_id,status,original_performance,optimized_performance,param_changes,top_params_index,top_paras
0,4,success,"[471.0, 570.0, 21.5]","[463.25, 574.74, 25.2]","[{'param_idx': 2, 'param_name': '化学元素含量过程的锰', ...","[2, 6, 0, 1, 5]","[锰, 铌, 碳, 硅, 钛]"
1,13,success,"[458.0, 562.0, 19.0]","[430.89, 541.12, 30.08]","[{'param_idx': 2, 'param_name': '化学元素含量过程的锰', ...","[2, 0, 6, 1, 5]","[锰, 碳, 铌, 硅, 钛]"
2,15,success,"[133.0, 275.0, 46.5]","[150.2, 279.9, 46.52]","[{'param_idx': 2, 'param_name': '化学元素含量过程的锰', ...","[2, 0, 5, 1, 12]","[锰, 碳, 钛, 硅, 卷取温度平均值]"
3,27,success,"[131.0, 280.0, 47.5]","[150.62, 281.16, 46.495]","[{'param_idx': 2, 'param_name': '化学元素含量过程的锰', ...","[2, 0, 5, 1, 12]","[锰, 碳, 钛, 硅, 卷取温度平均值]"
